In [13]:
import os, glob
import sys
import json
from PIL import Image
from collections import Counter

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import tifffile as tiff
import matplotlib.pyplot as plt
from tqdm import tqdm

import cv2

from sklearn.model_selection import KFold

sys.path.append("detection-wheel")

In [14]:
EPOCHS = 50

In [15]:
import os
import numpy as np
import torch
from PIL import Image


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, imgs, masks, transforms):
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = imgs#sorted(glob.glob('/home/viktor/Documents/kaggle/hubmap-2023/experiments/mask-rcnn/new-dataset/train/image/*.png'))
        self.masks = masks#sorted(glob.glob('/home/viktor/Documents/kaggle/hubmap-2023/experiments/mask-rcnn/new-dataset/train/mask/*.png'))

    def __getitem__(self, idx):
        # load images and masks
        img_path = self.imgs[idx]
        mask_path = self.masks[idx]
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path).convert('L')
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        #masks = (mask == obj_ids[:, None, None])
        #print((obj_ids[:, None, None]).shape)
        #masks = mask == obj_ids[:, None, None]
        masks = [np.where(mask== obj_ids[i, None, None],1,0) for i in range(len(obj_ids))]
        masks = np.array(masks)

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.nonzero(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        try:
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
            #print(area,area.shape,area.dtype)
        except:
            area = torch.tensor([[0],[0]])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        
        #print(masks.shape)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [16]:
import torchvision
from torchvision.models import list_models
detection_models = list_models(module=torchvision.models.detection)
detection_models

['fasterrcnn_mobilenet_v3_large_320_fpn',
 'fasterrcnn_mobilenet_v3_large_fpn',
 'fasterrcnn_resnet50_fpn',
 'fasterrcnn_resnet50_fpn_v2',
 'fcos_resnet50_fpn',
 'keypointrcnn_resnet50_fpn',
 'maskrcnn_resnet50_fpn',
 'maskrcnn_resnet50_fpn_v2',
 'retinanet_resnet50_fpn',
 'retinanet_resnet50_fpn_v2',
 'ssd300_vgg16',
 'ssdlite320_mobilenet_v3_large']

In [17]:
import torchvision
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models.resnet import ResNet50_Weights

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(weights="DEFAULT", weights_backbone=ResNet50_Weights.IMAGENET1K_V2)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)
    
    # model.rpn.anchor_generator.sizes = ((8, 16, 32, 64, 128,256),)

    return model

In [18]:
# model = get_model_instance_segmentation(2)
# model.rpn.anchor_generator.__dict__

In [19]:
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
        transforms.append(T.RandomIoUCrop())
        transforms.append(T.RandomZoomOut())
        transforms.append(T.RandomPhotometricDistort())
        transforms.append(T.ScaleJitter())
        transforms.append(T.RandomShortestSize())
    return T.Compose(transforms)

In [20]:
from engine import train_one_epoch, evaluate
import utils

In [21]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [22]:
n_imgs = len(glob.glob('/home/viktor/Documents/kaggle/hubmap-2023/experiments/mask-rcnn/new-dataset/train/image/*'))
n_imgs



1622

In [23]:
import sys

# Create a custom function to log output
def log_output(text):
    with open('output.log', 'a') as f:
        f.write(text)

# Redirect stdout to the custom log function
sys.stdout.write = log_output


In [24]:
kf = KFold(n_splits=5, shuffle=True, random_state=43)
for i, (train_index, test_index) in enumerate(kf.split(range(n_imgs))):
    if i!=0: continue
    all_imgs = sorted(glob.glob('/home/viktor/Documents/kaggle/hubmap-2023/experiments/mask-rcnn/new-dataset/train/image/*.png'))
    all_masks = sorted(glob.glob('/home/viktor/Documents/kaggle/hubmap-2023/experiments/mask-rcnn/new-dataset/train/mask/*.png'))
    all_imgs = np.array(all_imgs)
    all_masks = np.array(all_masks)
    train_img = all_imgs[train_index]
    train_mask = all_masks[train_index]
    val_img = all_imgs[test_index]
    val_mask = all_masks[test_index]
    dataset_train = PennFudanDataset(train_img, train_mask, get_transform(train=True))
    dataset_val = PennFudanDataset(val_img, val_mask, get_transform(train=False))
    train_dl = torch.utils.data.DataLoader(
        dataset_train, batch_size=4, shuffle=True, num_workers=os.cpu_count(), pin_memory=True, drop_last=True, collate_fn=utils.collate_fn)
    val_dl = torch.utils.data.DataLoader(
        dataset_val, batch_size=1, shuffle=False, num_workers=os.cpu_count(), pin_memory=True,collate_fn=utils.collate_fn)
    
    model = get_model_instance_segmentation(num_classes=2)
    model.to(device)
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.001)
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    # set linear warmup scheduler, with constant learning rate after warmup
    # scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.0001,
    #                                             steps_per_epoch=10, epochs=EPOCHS//10,
    #                                             pct_start=0.01)
    
    
    scheduler = torch.optim.lr_scheduler.LinearLR(optimizer,start_factor=0.001,
                                            end_factor=1,
                                            total_iters=30)
    
    for epoch in range(EPOCHS):
        train_one_epoch(model, optimizer, train_dl, device, epoch, print_freq=50)
        evaluate(model, val_dl, device=device)
        scheduler.step()
        model_path = f'ckpts/fold_{i}_epoch{epoch}.pth'
        torch.save(model.state_dict(), model_path)
        

/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None

Epoch: [0]  [  0/324]  eta: 0:25:13  lr: 0.000001  loss: 4.0384 (4.0384)  loss_classifier: 0.7254 (0.7254)  loss_box_reg: 0.1882 (0.1882)  loss_mask: 2.0480 (2.0480)  loss_objectness: 0.9909 (0.9909)  loss_rpn_box_reg: 0.0859 (0.0859)  time: 4.6698  data: 2.7855  max mem: 7785
Epoch: [0]  [ 50/324]  eta: 0:01:27  lr: 0.000001  loss: 3.4349 (3.7900)  loss_classifier: 0.7247 (0.7278)  loss_box_reg: 0.0890 (0.0982)  loss_mask: 1.1168 (1.1838)  loss_objectness: 1.3494 (1.5778)  loss_rpn_box_reg: 0.1402 (0.2024)  time: 0.2282  data: 0.0002  max mem: 12051
Epoch: [0]  [100/324]  eta: 0:00:59  lr: 0.000001  loss: 3.8681 (3.8713)  loss_classifier: 0.7302 (0.7286)  loss_box_reg: 0.1037 (0.1006)  loss_mask: 1.3184 (1.2058)  loss_objectness: 1.4582 (1.6302)  loss_rpn_box_reg: 0.1797 (0.2061)  time: 0.2098  data: 0.0002  max mem: 12051
Epoch: [0]  [150/324]  eta: 0:00:43  lr: 0.000001  loss: 4.5517 (3.9605)  loss_classifier: 0.7150 (0.7268)  loss_box_reg: 0.1353 (0.1052)  loss_mask: 1.1136 (1.2070

/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None

Epoch: [1]  [  0/324]  eta: 0:18:47  lr: 0.000051  loss: 3.8136 (3.8136)  loss_classifier: 0.7348 (0.7348)  loss_box_reg: 0.1101 (0.1101)  loss_mask: 2.0111 (2.0111)  loss_objectness: 0.8594 (0.8594)  loss_rpn_box_reg: 0.0982 (0.0982)  time: 3.4796  data: 3.0565  max mem: 12307


/home/viktor/miniconda3/envs/torch-env/lib/python3.9/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch: [1]  [ 50/324]  eta: 0:01:23  lr: 0.000051  loss: 3.5735 (3.6401)  loss_classifier: 0.5967 (0.6413)  loss_box_reg: 0.1055 (0.1038)  loss_mask: 1.0240 (1.0884)  loss_objectness: 1.4578 (1.5977)  loss_rpn_box_reg: 0.2214 (0.2089)  time: 0.2167  data: 0.0002  max mem: 12307
Epoch: [1]  [100/324]  eta: 0:00:57  lr: 0.000051  loss: 3.3972 (3.5183)  loss_classifier: 0.5128 (0.5889)  loss_box_reg: 0.1335 (0.1124)  loss_mask: 1.3673 (1.1864)  loss_objectness: 1.2676 (1.4436)  loss_rpn_box_reg: 0.1270 (0.1869)  time: 0.2063  data: 0.0002  max mem: 12307
Epoch: [1]  [150/324]  eta: 0:00:42  lr: 0.000051  loss: 2.8821 (3.3758)  loss_classifier: 0.4418 (0.5456)  loss_box_reg: 0.1033 (0.1131)  loss_mask: 0.6307 (1.0955)  loss_objectness: 1.2166 (1.4272)  loss_rpn_box_reg: 0.1714 (0.1944)  time: 0.2135  data: 0.0002  max mem: 12307


KeyboardInterrupt: 

In [ ]:
all_indices = np.arange(n_imgs)
# take random 1400 images for training
train_index = np.random.choice(all_indices, size=1400, replace=False)
# take the rest for validation
test_index = np.setdiff1d(all_indices, train_index)



In [ ]:
# check if train_index and test_index are mutually exclusive
len(np.intersect1d(train_index, test_index))

In [ ]:
# import sys

# # Create a custom function to log output
# def log_output(text):
#     with open('output.log', 'a') as f:
#         f.write(text)

# # Redirect stdout to the custom log function
# sys.stdout.write = log_output


In [ ]:


# all_imgs = sorted(glob.glob('/home/viktor/Documents/kaggle/hubmap-2023/experiments/mask-rcnn/new-dataset/train/image/*.png'))
# all_masks = sorted(glob.glob('/home/viktor/Documents/kaggle/hubmap-2023/experiments/mask-rcnn/new-dataset/train/mask/*.png'))
# all_imgs = np.array(all_imgs)
# all_masks = np.array(all_masks)
# train_img = all_imgs[train_index]
# train_mask = all_masks[train_index]
# val_img = all_imgs[test_index]
# val_mask = all_masks[test_index]
# dataset_train = PennFudanDataset(train_img, train_mask, get_transform(train=True))
# dataset_val = PennFudanDataset(val_img, val_mask, get_transform(train=False))
# train_dl = torch.utils.data.DataLoader(
#     dataset_train, batch_size=4, shuffle=True, num_workers=os.cpu_count(), pin_memory=True, drop_last=True, collate_fn=utils.collate_fn)
# val_dl = torch.utils.data.DataLoader(
#     dataset_val, batch_size=1, shuffle=False, num_workers=os.cpu_count(), pin_memory=True,collate_fn=utils.collate_fn)

# model = get_model_instance_segmentation(num_classes=2)
# model.to(device)
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.Adam(params, lr=0.00003)
# # optimizer = torch.optim.Adam(params, lr=0.0001, weight_decay=1e-6)
# # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
# # set linear warmup scheduler, with constant learning rate after warmup
# scheduler = torch.optim.lr_scheduler.LinearLR(optimizer,start_factor=0.01,
#                                             end_factor=1,
#                                             total_iters=10)

# for epoch in range(EPOCHS):
#     train_one_epoch(model, optimizer, train_dl, device, epoch, print_freq=50)
#     evaluate(model, val_dl, device=device)
#     scheduler.step()
#     model_path = f'ckpts/fold_{0}_epoch{epoch}.pth'
#     torch.save(model.state_dict(), model_path)
    